<a href="https://colab.research.google.com/github/anni1st/Anomaly-Detection-VM-/blob/main/anomaly_detection_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import packages

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/datadummy_CPU&memory.csv')

In [ ]:
memory = pd.read_csv("/content/drive/MyDrive/datadummy_CPU&memory.csv")
memory.head()

,Vm ID,CPU Used(%),Disk IO Read(KiB),Disk IO Write(KiB),Memory Used(KiB),Create At
0,87aec8ce-5fe8-4d7b-a05e-656edfcef22d,96%,1045,13610354,92209370,2020-01-03TH:00:24.000000Z
1,da09925d-e999-4b6c-a3c0-fb70298000ae,9%,1499,14909641,98758823,2020-01-03TH:00:59.000000Z
2,a81be7e5-cd69-42aa-a631-c5cd421ccd57,2%,299,27729627,93081576,2020-01-04TH:12:24.000000Z
3,0d3f3331-b725-4ad9-bdad-36fe2248a0e9,44%,843,23615556,95596292,2020-01-04TH:34:46.000000Z
4,402cb178-2c4b-4f3b-a759-3d37e6d29582,5%,762,11647904,93886601,2020-01-04TH:40:05.000000Z


In [ ]:
main_df = df[['Vm ID', 'Memory Used(KiB)']]
main_df.head()

,Vm ID,Memory Used(KiB)
0,87aec8ce-5fe8-4d7b-a05e-656edfcef22d,92209370
1,da09925d-e999-4b6c-a3c0-fb70298000ae,98758823
2,a81be7e5-cd69-42aa-a631-c5cd421ccd57,93081576
3,0d3f3331-b725-4ad9-bdad-36fe2248a0e9,95596292
4,402cb178-2c4b-4f3b-a759-3d37e6d29582,93886601
